# 분류 전 과정 연습
- https://dataq.goorm.io/exam/116674/%EC%B2%B4%ED%97%98%ED%95%98%EA%B8%B0/quiz/3
- https://ha2juo.tistory.com/5
- https://blog.naver.com/PostView.naver?blogId=da0097&logNo=222390408292&categoryNo=0&parentCategoryNo=0
- https://m.blog.naver.com/da0097/222582321859

In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# 사용자 코딩

# 1. 데이터 전처리
# Train set---------------------------------------------------------------------------------
# cust_id 기준으로 inner join
train_set = pd.merge(X_train, y_train, how = 'inner', on = 'cust_id')

# X, y 다시 나누기
X_train = train_set[train_set.columns[:-1]]
y_train = train_set[['gender']]

# 결측치 확인
# print(X_train.isnull().sum())
# print(y_train.isnull().sum())

# 결측치 대체
X_train = X_train.fillna(0)
# print(X_train.isnull().sum())

# cust_id 제외
X_train = X_train[X_train.columns[1:]]
# print(X_train.describe())

# 수치형 데이터 따로 빼서 MinMaxScaling 하기
from sklearn.preprocessing import MinMaxScaler
X_train_minmax1 = X_train[X_train.columns[:3]]
X_train_minmax2 = X_train[X_train.columns[5:]]
X_train_minmax = pd.concat([X_train_minmax1, X_train_minmax2], axis = 1)

scaler = MinMaxScaler()
scaler.fit(X_train_minmax)
minmax_scaled_X_train = scaler.transform(X_train_minmax)
X_train_scaled_num = pd.DataFrame(columns = X_train_minmax.columns,
						 data = minmax_scaled_X_train)

# 범주형 데이터 따로 빼서 one-hot encoding
X_train_onehot = X_train[X_train.columns[3:5]]
X_train_onehot = pd.get_dummies(X_train_onehot)


# 스케일링 데이터 붙이기!
scaled_X_train = pd.concat([X_train_scaled_num, X_train_onehot], axis = 1)
# Train set---------------------------------------------------------------------------------

# Test set----------------------------------------------------------------------------------
# 결측치 확인
# print(X_test.isnull().sum())
X_test = X_test.fillna(0)

# cust_id 제외
X_test_remove_id = X_test[X_test.columns[1:]]

# 수치형 데이터 따로 빼서 MinMaxScaling 하기
from sklearn.preprocessing import MinMaxScaler
X_test_minmax1 = X_test_remove_id[X_test_remove_id.columns[:3]]
X_test_minmax2 = X_test_remove_id[X_test_remove_id.columns[5:]]
X_test_minmax = pd.concat([X_test_minmax1, X_test_minmax2], axis = 1)

scaler = MinMaxScaler()
scaler.fit(X_test_minmax)
minmax_scaled_X_test = scaler.transform(X_test_minmax)
X_test_scaled_num = pd.DataFrame(columns = X_test_minmax.columns,
						 data = minmax_scaled_X_test)
# print(X_test_scaled_num)

# 범주형 데이터 따로 빼서 one-hot encoding
X_test_onehot = X_test_remove_id[X_test_remove_id.columns[3:5]]
X_test_onehot = pd.get_dummies(X_test_onehot)


# 스케일링 데이터 붙이기!
scaled_X_test = pd.concat([X_test_scaled_num, X_test_onehot], axis = 1)

# Test set----------------------------------------------------------------------------------

### 새로운 이슈: Train set의 원핫인코딩 컬럼 수가 Test set의 그것보다 하나 더 많다!
# print(X_train_onehot)
# print(X_test_onehot)
# for col in X_train_onehot.columns:
# 	if col not in X_test_onehot.columns:
# 		print(col)

# [주구매상품_소형가전] 열을 test에 추가해주자!
scaled_X_test['주구매상품_소형가전'] = 0
# print(scaled_X_test)



# # 2. 모델 생성 및 평가
import warnings
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

warnings.filterwarnings('ignore')

clfs = [['LOGIS', LogisticRegression()],
				['RF', RandomForestClassifier()],
				['XGB', XGBClassifier()]]

for (name, model) in clfs:
	score = cross_val_score(model, scaled_X_train, y_train, cv=3)
	print("%s's score: %f" % (name, score.mean()))
	print("\n")

# 3. RandomForest 모델 채택 및 predict y
final_model = RandomForestClassifier()
final_model.fit(scaled_X_train, y_train)

# confusion matrix 생성 (Train data set에 대해)
predict_gender = final_model.predict(scaled_X_train)
cf = confusion_matrix(y_train, predict_gender)
print(cf)

# 확률 예측 (predict_proba)
predict_proba = final_model.predict_proba(scaled_X_test)

# '남자'일 확률만 추출
proba_men = []
for p in predict_proba:
	proba_men.append(p[1])
# print(proba_men)

# '남자'로 예측한 확률을 test data에 'cust_id' 열과 붙이기
result = X_test[['cust_id']]
result['gender'] = proba_men
print(result)

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
pd.DataFrame({'cust_id': X_test.cust_id, 'gender': proba_men}).to_csv('25169.csv', index=False)
